<a href="https://colab.research.google.com/github/jinsusong/21-study-paper-review/blob/main/Huggingface_distilbert_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 데이터 증폭 

In [ ]:
!pip install numpy requests nlpaug
!pip install transformers

import pandas as pd
import numpy as np
import nlpaug.augmenter.word as nlpaw
from tqdm import tqdm



def augment_sentence(sentence, aug, num_threads):
    """""""""
    Constructs a new sentence via text augmentation.
    
    Input:
        - sentence:     A string of text
        - aug:          An augmentation object defined by the nlpaug library
        - num_threads:  Integer controlling the number of threads to use if
                        augmenting text via CPU
    Output:
        - A string of text that been augmented
    """""""""
    return aug.augment(sentence, num_thread=num_threads)
    


def augment_text(df, aug, num_threads, num_times):
    """""""""
    Takes a pandas DataFrame and augments its text data.
    
    Input:
        - df:            A pandas DataFrame containing the columns:
                                - 'comment_text' containing strings of text to augment.
                                - 'isToxic' binary target variable containing 0's and 1's.
        - aug:           Augmentation object defined by the nlpaug library.
        - num_threads:   Integer controlling number of threads to use if augmenting
                         text via CPU
        - num_times:     Integer representing the number of times to augment text.
    Output:
        - df:            Copy of the same pandas DataFrame with augmented data 
                         appended to it and with rows randomly shuffled.
    """""""""
    
    # Get rows of data to augment
    to_augment = df[df['isToxic']==1]
    to_augmentX = to_augment['comment_text']
    to_augmentY = np.ones(len(to_augmentX.index) * num_times, dtype=np.int8)
    
    # Build up dictionary containing augmented data
    aug_dict = {'comment_text':[], 'isToxic':to_augmentY}
    for i in tqdm(range(num_times)):
        augX = [augment_sentence(x, aug, num_threads) for x in to_augmentX]
        aug_dict['comment_text'].extend(augX)
    
    # Build DataFrame containing augmented data
    aug_df = pd.DataFrame.from_dict(aug_dict)
    
    return df.append(aug_df, ignore_index=True).sample(frac=1, random_state=42)
    

    
# Define nlpaug augmentation object 
aug10p = nlpaw.ContextualWordEmbsAug(model_path='bert-base-uncased', aug_min=1, aug_p=0.1, action="substitute")

# Upsample minority class ('isToxic' == 1) to create a roughly 50-50 class distribution
balanced_df = augment_text(downsampled_df, aug10p, num_threads=8, num_times=3)

### 텍스트 토큰화

In [ ]:
# 사전 훈련된 모델을 선택하고 나면 사람이 읽을 수 있는 텍스트 문자열을 모델이 해석할 수 있는 형식으로 변환해야 함
# 이 프로세스를 토큰화 라고 하며 직관적인 Hugging Face API를 사용하면 단어와 문장 → 토큰 시퀀스 → 텐서로 변환되어 모델에 제공될 수 있는 숫자 시퀀스를 매우 쉽게 변환할 수 있음
# 토크나이저가 토큰화 체계에 [CLS] 및 [SEP]와 같은 특수 토큰을 포함하는 방법을 참조
# 선택한 모델에 특정한 토크나이저 객체. 에서 사용하는 토크나이저를 가져오기 위해 distilbert-base-uncased모델의 이름을 DistilBertTokenizerFast. 클래스의 .from_pretrained()메서드에 전달합니다 

from transformers import DistilBertTokenizerFast

# Instantiate DistilBERT tokenizer...we use the Fast version to optimize runtime
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
# 토크아니저 객체를 인스턴스화하면 토크나이저의 .batch_encode_plus() 방법을 사용하여 훈련, 검증 및 테스트 세트를 일괄적으로 인코딩할 수 있습니다. 


# Define the maximum number of words to tokenize (DistilBERT can tokenize up to 512)
MAX_LENGTH = 128


# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts' corresponding encodings and attention masks that are ready to be fed into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []

    # 인수 설정
    # max_length → 주어진 텍스트에서 토큰화할 최대 단어 수를 제어합니다.
    # padding → '가장 긴'으로 설정하면 배치에서 가장 긴 순서로 채웁니다.
    # truncation→ True 이면 max_length 에서 설정한 값에 따라 텍스트를 자릅니다
    # return_attention_mask→ True 이면 attention mask를 반환, 선택 사항이지만 attention mask는 가중치를 높게 봐야하는 토큰과 무시해야 하는 토큰(패딩의 경우)을 모델에 알려준다. 모델에 대한 입력으로 주의 마스크를 포함하면 모델 성능이 향상될 수 있다.
    # return_token_type_ids→ True 이면 토큰 유형 ID 를 반환,  토큰 ID가 입력의 한 시퀀스가 ​​끝나고 다른 시퀀스가 ​​시작하는 위치를 모델에 알려주기 때문에 입력으로 여러 시퀀스가 ​​필요한 부분에 작업이 필요합니다(예: 질문 응답에는 '질문' 및 '답변' 시퀀스가 ​​필요함). 그러나 분류 작업에는 입력으로 하나의 시퀀스만 필요하기 때문에 이것은 선택 사항이다..
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='longest', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    
    
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.tolist())

# Encode X_valid
X_valid_ids, X_valid_attention = batch_encode(tokenizer, X_valid.tolist())

# Encode X_test
X_test_ids, X_test_attention = batch_encode(tokenizer, X_test.tolist())


#  batch_encode 함수는 다음을 반환한다.
# input_ids → 일련의 숫자로 인코딩된 텍스트의 단어.
# attention_mask→ input_ids주의를 기울여야 할 숫자 와 무시해야 할 숫자를 모델에 알려주는 이진 시퀀스 (패딩의 경우).
# input_ids 및 attention_mask  둘 다 Tensorflow tf.Tensor 객체로 변환되어 모델에 입력으로 쉽게 제공될 수 있다.

### 모델 아키텍처 정의

- 훈련, 검증 및 테스트 세트를 인코딩했으므로 이제 모델 아키텍처를 정의할 차례
- 기본 모델로 DistilBERT를 사용할 것이므로  Hugging Face 라이브러리에서 distilbert-base-uncased 가져오기


- Hugging Face API가 DistilBERT의 구성 클래스에서 여러 인수를 변경하여 기본 모델 아키텍처를 조정할 수 있는 config 옵션을 제공한다

- DistilBERT의 구성 클래스를 수정한 후, 모델 이름과 구성 개체를 TFDistilBertModel클래스 의 .from_pretrained()메서드에 전달하여 특정 헤드 없이 기본 DistilBERT 모델을 TFDistilBertForSequenceClassification 인스턴스화 할 수 있다.

- DistilBERT의 사전 훈련된 가중치는 우리 모델의 기초 역할을 하기 때문에 모델이 추가된 분류 계층에 대해 합리적인 가중치를 학습하기 시작할 때 훈련의 초기 단계에서 업데이트를 방지하고 보존하고자 한다.

- DistillBERT의 사전 훈련된 가중치를 일시적으로 고정하려면 각 DistilBERT의 레이어에 대해 layer.trainable = False 설정 하고 나중에 모델 성능이 수렴되면 layer.trainable = True 설정하여 고정을 해제할 수 있다.

In [ ]:
# 기본 모델 초기화 
from transformers import TFDistilBertModel, DistilBertConfig

DISTILBERT_DROPOUT = 0.2
DISTILBERT_ATT_DROPOUT = 0.2
 
# Configure DistilBERT's initialization
config = DistilBertConfig(dropout=DISTILBERT_DROPOUT, 
                          attention_dropout=DISTILBERT_ATT_DROPOUT, 
                          output_hidden_states=True)
                          
# The bare, pre-trained DistilBERT transformer model outputting raw hidden-states 
# and without any specific head on top.
distilBERT = TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)

# Make DistilBERT layers untrainable
for layer in distilBERT.layers:
    layer.trainable = False